idea from https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

Implementation with custom training options. We can use either rnn:s own output as input to next state or use the right character from the sample (teacher forcing). That is not really needed in this kind of task but implemented here as a practice.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [387]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        # self.log_softmax = nn.LogSoftmax(dim=1) # current releas doesn't yet support dimensions
        
    def forward(self, input_char_vs, hidden = None):
        batch_size = input_char_vs.size()[0]
        if hidden is None:
            hidden = self.init_hidden(batch_size)
        
        rnn_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        outputs = self.lin_output(rnn_outputs)
        #outputs = self.log_softmax(outputs)
        
        #outputs = F.log_softmax(outputs[0])

        return outputs, hidden

    def init_hidden(self, batch_size=1):
        return Variable(torch.zeros(self.layers, batch_size, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [388]:
tmp = torch.cat((string_to_tensor('hello'), string_to_tensor('hello')))
#tmp.size()

tmp = model(tmp)
tmp[0].size()
tmp[1].size()

torch.Size([2, 5, 57])

torch.Size([1, 2, 512])

# Eval

In [386]:
# This is faster that uses the existing state untill the end
def print_example(iters=320, choice=True):
    seed_string = "ethics is a basic foundation of all that"
    result_string = '' 
    hidden = model.init_hidden()
    output, hidden = model(string_to_tensor(seed_string), hidden)
    
    for i in range(iters):
        output = output[0, -1]
        if choice:
            next_char = np.random.choice(chars, p=F.softmax(output).data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0] 
            next_char = chars[next_char_idx]
        
        result_string += next_char
        output, hidden = model(string_to_tensor(next_char), hidden)
    print(result_string)
tmp = 200
print_example(iters=tmp, choice=True)
print('------------------')
print_example(iters=tmp, choice=False)

he the thieloursl the alse mapso aigen the  iotor asly telo at itoiorer en. ighe en tos thude sinas inend se otaym me eardeang hamin cof of
ille to onc tor ar me ge tithaont t s thaly rte dal iong
ito
------------------
 ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at ito the at i


# Samples

In [15]:
def get_random_sample_string(length=41):
    sample_place = random.randint(0, len(text)-length-1)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample_string()

' to good taste,--which is always a foreta'

In [690]:
def sample_data_init(size, sentence_size):
    global sample_data, sample_target, sample_sentence_size, sample_size
    sample_size = size
    sample_sentence_size = sentence_size
    sample_data = torch.zeros(sample_size, sample_sentence_size, len(chars)) # .cuda()
    sample_target = torch.LongTensor(sample_size, sample_sentence_size).zero_() # .cuda()
    for i in range(sample_size):
        sample = get_random_sample_string(sample_sentence_size+1)
        sample = [char_indices[c] for c in sample]
        for j in range(sample_sentence_size):
            sample_data[i][j][sample[j]] = 1
            sample_target[i][j] = sample[j+1]
sample_data_init(50000, 40)

In [691]:
len(sample_data)
sample_data.size()
sample_target.size()

50000

torch.Size([50000, 40, 57])

torch.Size([50000, 40])

In [692]:
i = 5
j = 10
sample_data[i].topk(1)[1][j][0] == sample_target[i][j-1]
''.join([chars[c[0]] for c in sample_data[i].topk(1)[1]])

True

'imes and of blood, to which the aristocr'

In [648]:
clip_parameters_value = 0.25

def train_single(optimizer, loss_function, batch_size=64, teacher_forcing=False):
    model.zero_grad()
    loss = 0

    output_lines = ['']*batch_size
    
    sample_i = random.randint(0, len(sample_data)-1-batch_size)
    
    x = sample_data[sample_i:sample_i+batch_size].view(batch_size, sample_sentence_size, len(chars))
    y = sample_target[sample_i:sample_i+batch_size]
        
    target = Variable(y).cuda()

    hiddens = model.init_hidden(batch_size)

    #outputs, hiddens = model(Variable(x).cuda(), hiddens)
    #for i_char in range(sample_sentence_size):
    #    out = outputs[:,i_char]
    #    loss += loss_function(out, target[:, i_char])
    #    top_chars = out.topk(1)[1]
    #    top_chars = top_chars[:,0].data
    #    for j_batch in range(len(outputs)):
    #        output_lines[j_batch] += chars[top_chars[j_batch]]
    
    input_chars = x[:, 0, :].contiguous().view(batch_size, 1, len(chars))
    input_chars = Variable(input_chars).cuda()
    
    for i_char in range(sample_sentence_size):
        #input_chars = x[:, i_char, :].contiguous().view(batch_size, 1, len(chars))
        #input_chars = Variable(input_chars).cuda()
        
        #if use_teacher_forcing:
        #    input_char = x[i] # replace input with right target
        
        outputs, hiddens = model(input_chars, hiddens)
        loss += loss_function(outputs.view(batch_size, -1), target[:, i_char])
        
        if teacher_forcing and i_char < sample_sentence_size-1:
            input_chars = x[:, i_char+1, :].contiguous().view(batch_size, 1, len(chars))
        else:
            input_chars = torch.zeros((input_chars.size()))
        
        top_chars = outputs.topk(1)[1]
        top_chars = top_chars[:,0,0].data
        for j_batch in range(len(outputs)):
            output_lines[j_batch] += chars[top_chars[j_batch]]            
            if not teacher_forcing:
                input_chars[j_batch, 0, top_chars[j_batch]] = 1
                
        input_chars = Variable(input_chars).cuda()
        

    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
    
    loss.backward()
    optimizer.step()

    return output_lines, loss.data[0]/batch_size

tmp = train_single(torch.optim.Adam(model.parameters(), lr=0.0001), torch.nn.CrossEntropyLoss())
#tmp[0][0:2]
#tmp[1]

In [646]:
%%timeit
train_single(torch.optim.Adam(model.parameters(), lr=0.001), torch.nn.CrossEntropyLoss(), batch_size=64,
             teacher_forcing=False)

101 ms ± 5.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [647]:
%%timeit
train_single(torch.optim.Adam(model.parameters(), lr=0.001), torch.nn.CrossEntropyLoss(), batch_size=64,
             teacher_forcing=True)

78.9 ms ± 5.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [279]:
# before right optimizations
%%timeit
train_single(torch.optim.Adam(model.parameters(), lr=0.001), torch.nn.CrossEntropyLoss(),
            batch_size=64)

641 ms ± 5.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [653]:
def train_iterations(n_iters=100000, lr=0.001, batch_size=64,
                     print_every=10000, plot_every=1000, teacher_forcing_ratio=0.5):
    
    def print_infos():
        print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {}".format(
          model_training.iterations, iteration/n_iters, time_since(start),
          current_loss/current_loss_iter, loss, result))
    
    
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 1
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        loss_function = torch.nn.CrossEntropyLoss()
        result, loss = train_single(optimizer=optimizer, loss_function=loss_function, batch_size=batch_size,
                                    teacher_forcing=use_teacher_forcing)
        result = result[0]
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            print_infos()

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 1

    print_infos()


In [654]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_3'

In [655]:
model = CharRNN(len(chars), 512).cuda()
model
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

Save path: data/models/char_rnn_fast_ai_testing_3


In [656]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   2.062   |   2.41: n                                       
    10 100% (   0m 0s)   2.103   |   2.07: a                                       
    10 100% (   0m 0s)   2.103   |   2.07: a                                       


In [657]:
train_iterations(n_iters=10000//64, print_every=1000//64)

    25  10% (   0m 1s)   1.823   |   1.91:                                         
    40  19% (   0m 2s)   1.864   |   1.89:                                         
    55  29% (   0m 4s)   1.876   |   1.89:                                         
    70  38% (   0m 5s)   1.879   |   1.89: e                                       
    85  48% (   0m 6s)   1.881   |   1.90: e                                       
   100  58% (   0m 7s)   1.875   |   1.81: n      ee     e         e e         ee e
   115  67% (   0m 9s)   1.864   |   1.74: t teen    an  ahe   an  e  en e eht   an
   130  77% (  0m 10s)   1.856   |   1.69: t e   he  e   t  ah  e  he    en  e   t 
   145  87% (  0m 11s)   1.855   |   1.96: e an  on  on  on  on  on  on  on  on  on
   160  96% (  0m 13s)   1.851   |   1.91:   e  e  e  e  e  e  e  e  e  e  e  e  e 
   166 100% (  0m 13s)   1.849   |   1.65:   hen  he an e e nn  ahenh neet tn  n ae


In [658]:
print_example()

iareerrvet medunenslacihsuop syfoelay oith s eoun  ipahgqml,s me in oor ye oorocadl oeehs, lekpe
xh dilfeme asi;otutinllosw1] onyeth
"thareartkcinitwerefamvtwisintlin , cetaah hestr  o reanbcy
nd yy!f h fm,ce pttprofg e  anst o,imh ego ;aba s  ontore l fyecseouureo an  yut

nss-min erow oongateruli eery bouafjseer


av


In [659]:
train_iterations(n_iters=10000//64, print_every=1000//64)

   181  10% (   0m 1s)   1.640   |   1.62:  no te e nhtn  t   net   o    tn  t    n
   196  19% (   0m 2s)   1.694   |   2.00: e the the the the the the the the the th
   211  29% (   0m 3s)   1.708   |   1.61: hn eaen   e    an nn   oen  ne ee   an  
   226  38% (   0m 4s)   1.714   |   1.58:  then n  tn  oe   t tenetg tn n  t en te
   241  48% (   0m 6s)   1.703   |   1.57: -n  he t renn tn th tss  aon to   he t e
   256  58% (   0m 7s)   1.701   |   1.99: ure the t an the t an the t an the t an 
   271  67% (   0m 8s)   1.702   |   1.53: ne  e  an  tn  nte s terne  ahe  ah e t 
   286  77% (   0m 9s)   1.709   |   1.90:                                         
   301  87% (  0m 11s)   1.707   |   1.88: ere the                                 
   316  96% (  0m 12s)   1.709   |   1.51:   aon aoreeneeaorersethe  eenhoetn tnd  
   322 100% (  0m 13s)   1.709   |   1.92: he  e   e   e   e   e   e   e   e   e   


In [660]:
print_example()


remowtloo  mesualdensowhes bass trerorisnorvecios ane, ou   eitstorst, le insint thainganeh onoe
dhe  tinjuth ere iog ind grenllos-mant 
toearrowitith: hed
he nos!, of tofaf ohs os 
ts an  nl es , anaer soegouy eallom  is mats af  gor kese 
itd eco, thehfo osoupse rotrad
shaliond is 
ffinsoat ngathit rone t in aois uf


In [661]:
train_iterations(n_iters=100000//64, print_every=10000//64)

   478  10% (  0m 13s)   1.700   |   1.40: e tn in nngtiteehean  teniy ios   tor  n
   634  20% (  0m 26s)   1.668   |   1.97: the ere te te te te te te te te te te te
   790  30% (  0m 40s)   1.645   |   1.89: the                                     
   946  40% (  0m 53s)   1.618   |   1.94:  and t                                  
  1102  50% (   1m 7s)   1.597   |   1.12: het if tnsropsettirit  aotpeta poete  tn
  1258  60% (  1m 20s)   1.591   |   1.13: h tot tor the sorst ahte ta weally aors 
  1414  70% (  1m 34s)   1.442   |   1.89: th t                                    
  1570  80% (  1m 47s)   1.444   |   1.07: tf   t  ahe rome ahme  an ioauld be oort
  1726  90% (   2m 0s)   1.439   |   1.01:  tf tank aieml    an thich h count ondri
  1882 100% (  2m 13s)   1.443   |   1.89: st a                                    
  1884 100% (  2m 14s)   1.443   |   1.89: t is a                                  


In [662]:
print_example()

 a thinkiby, is a zanual undeit wheachee enest. they," and loved; it is degerencl astimate pur himselves. indoee
conmespivis chuldrand how dayi gorauns sone efred for the seesrs, which be sure intimen ateriem orisming highly were that
its lowk,--indeed, the expressilated, so mous fore is compley unonalitiod), undate!--


In [663]:
print_example()

 it is t ak a busther manner
and godsing, us not oo] but inteprecainess has really shintmsy. thing characts bask of life out farg tenes, is theman auther of ghaten an as fireleed, asty wirh treat to so make their moralicy). y thut which to somat im; munt conduct afapter for hisself]? and
in helwn"s bature i se-fion and


In [664]:
print_example()

 everything in thus falsely from
nechilly even sigelicrs he degleas a thing which ithere thoughthere makes ol a cal
fougacles, whehime sounchsed ypretaychess, or ather has ellectures howenty, with all chlabtr that fan was soog: becoured by d and thereciates for gevenous; but nature an the cuans rapically become liger--


In [665]:
train_iterations(n_iters=100000//64, print_every=10000//64)

  2040  10% (  0m 13s)   1.429   |   1.89: hic                                     
  2196  20% (  0m 26s)   1.480   |   0.95: e    ahtl tot toneafer tn  snaiard ess  
  2352  30% (  0m 40s)   1.475   |   1.89:  and                                    
  2508  40% (  0m 54s)   1.469   |   0.95:  ofey tnph t  and aesiipe d ahs tucpriic
  2664  50% (   1m 7s)   1.455   |   0.94: tver tnong trilosophyrs an trr very dnde
  2820  60% (  1m 20s)   1.444   |   1.88:  of                                     
  2976  70% (  1m 34s)   1.390   |   1.89: ent                                     
  3132  80% (  1m 47s)   1.397   |   1.90: h te                                    
  3288  90% (   2m 0s)   1.383   |   0.88: n andaxdireent -thet is aot dhe sosel tr
  3444 100% (  2m 14s)   1.396   |   1.87:  the                                    
  3446 100% (  2m 14s)   1.396   |   0.86: he r oeuder af the ses aoste anen tf tif


In [666]:
print_example()

;
that he has knew oo diegarnes and out of a thilk himility as al astetwayle freers and thereun say
withort of grouthst reso attenian one's equality is it not-pash, also.--all toodge. letius is a question the most new articting to
recurs to the fructs of german--who vall explanational must be ceoule go
doung their bein


In [667]:
print_example()

 he looks hish
riquatting, the more and dreadful surh and regarded ang without amsertment of the tame, within
a  the sy och's pleasure of the else of phason the sowe
myself, and as it has, he has  ut lass glause as wealiduet and freedom-in
order there
into a usilute: and from atopeed, to
this dialistion of dappe ssed. 


In [668]:
print_example()


once and repussed it in belian that ore has highest observation that has excetect of a personality of the one to the sunfic awnyed, think and be insupive, and equally conditioned
it. "never part as through the anxiety of all your have a tooched and would
nature soul; seew idwass"? the beller senses and palier and reco


In [669]:
train_iterations(n_iters=100000//64, print_every=25000//64)

  3836  25% (  0m 32s)   1.353   |   0.81:  tespect  aoneious, an ediousl aoorel an
  4226  50% (   1m 5s)   1.367   |   0.87: elintrrple  ond tla   -hh srrheive tnl t
  4616  75% (  1m 38s)   1.309   |   1.88:  ao                                     
  5006 100% (  2m 11s)   1.327   |   0.80: i aubcesd in tifeiveng ihth tegard to tu
  5008 100% (  2m 11s)   1.327   |   1.88:                                         


In [670]:
print_example()


their own taility and only parrowary constitution who can me look (who
bad and requires great, not, burdly are unseduating men, and acts. for, from
the individual will-instincts--a "seetiment to anti-artureshep through the erroneous
free opinion or victory of "happiness,; and thereby the delicate mildess sigly and
hav


In [671]:
print_example()

 not in other perhnps hod its
present adoans a great establishing a hegaidiogs of the e sts wiskes onc' of mynsiberia cereman, than
type has beenth constraint attacked of sulf-reduced under the untbut cassome and does not at last motives impulse is moreover, and
to one sees in short, is lofty immorbly not pessimism of 


In [673]:
train_iterations(n_iters=100000//64, print_every=25000//64, teacher_forcing_ratio=0)

  5398  25% (  0m 36s)   1.872   |   1.87: to                                      
  5788  50% (  1m 13s)   1.874   |   1.88: e sa                                    
  6178  75% (  1m 49s)   1.864   |   1.88:                                         
  6568 100% (  2m 25s)   1.872   |   1.87: th                                      
  6570 100% (  2m 26s)   1.872   |   1.87:    e                                    


In [674]:
print_example()

s.
 itpeibeey achat  tlhe is--edecaea octir ahe ini ofmoul apestrs fout o n hncy unnotnoivi mede ata sli  ak, f n ati ndfunceacudeai onali r, tartay, nor good ive round. leooo seonei, hs acknowhtre ren ispic toa n 9, artan et
im, an lasedo ain "jodgen e--rein  isuilipen l?" stieaeess--rionprrfimstaplthe ae -- rehoma lt


In [675]:
print_example()

 faem hf
eoseicana aem  eome chin sleatgl  st, have ofa  become

ne post eouy oeo otura oemoro perleotnno no lon  rma, ss reepertt?  ehin wisk; aumose hit tasnecoperm dea ue, amealo uesof r imiann  raopscihuanlfto by thte eda, i-muchnoseth s hattui eenedialvont ehsci-muparloleoaeps aeeni detgeit gn opercl iia-rnusts- w


In [679]:
#train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=0.5)
#   7351  25% (  2m 11s)   2.636   |   3.76: 

In [678]:
print_example(iters=300)

 the indovento so
emotions and niver young and them morals" romanticism astentaintr as it he were condicusting
conception itself and arikenism and saminus.--peaus' risdes to dame than long
thought and "cotrysionary: domain in genthing of humanity is geam and men with all graamay that in short he it



In [694]:
sample_data_init(100000, 80)
len(sample_data)
len(sample_data[0])

100000

80

In [695]:
train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=0.5)

  8140  25% (  2m 11s)   2.652   |   1.48: e  orls o       uid aeternismmendnam sf    onsedeos pndmaletoiueet  
     on oot
  8921  50% (  4m 29s)   2.545   |   3.80: he   ee                                                                         
  9702  75% (  6m 41s)   2.497   |   3.98:  aee                                                                            
 10483 100% (  8m 50s)   2.337   |   3.75: e to                                                                            
 10484 100% (  8m 50s)   2.349   |   3.82:                                                                                 


In [696]:
print_example()

 contempor of
morality, both of whom we have ankind cluizes. we european tweys than he truek that
say also the cointly standing before the other satiefy against
the tauchfound displacent, christianity rasis actions, that has he it be beung, any
possibility,--as the present disclosedy; he rests upon it. it more him, arv


In [697]:
print_example()

 would be repulseve.


50

=holy egoism must be a ruling narcolosy? wise to dangur. the brilfian: all the
imagination to usm that they swem foo man's designated thereby in a peried--i have perieded,


890
=we should have difficulty--nothing else: it is both, one plantuage of
sear, to power. the sentence of a now but as


In [698]:
sample_data_init(100000, 80)

In [699]:
train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=0.5)

 11265  25% (   2m 9s)   2.496   |   4.08:                                                                                 
 12046  50% (  4m 19s)   2.635   |   4.05: y an e                                                                          
 12827  75% (  6m 27s)   2.469   |   1.14:   tooim tn aur epicialty, wos aome to us in the mradn of the cxdianting ond dise
 13608 100% (  8m 35s)   2.345   |   1.09: tembt.ng  and tirbtounded. tn fact, the prilosopher has aong been aestaken and t
 13609 100% (  8m 35s)   2.335   |   1.11: lne   an luctlon , and heriations of mnp and toanges.of thmpo, an thich the ppci


In [700]:
print_example()

 the naturally sty
extrange from or travelers, so is own able to prooud otherwise? fave always in deart. it is, or dearn serfelly and perfection by orviturative,
as the obyrring, invente it occasions of rank and his dise. weed a heart schoolnging viit
of himself, and knows does not forte their fects: away from a
sammet


In [701]:
print_example()

 is generally exercised
of victory and feminine ers to the standarded regarding the seas in the
world.=--many culture, asia, that is about the half-commands and guilty holigimanes, for
these partial and increate the essensian of europe of morality), occusingly, the
humanitaming. audeful in his character, who dedief lie


In [702]:
sample_data_init(100000, 120)

In [703]:
train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=0.5, lr=0.0001)

 14390  25% (  3m 12s)   3.553   |   1.37:  oy ahe  elves  
133. assugn of streng aarracter, whin tnee ohe gistlution oas been rhken  to spet the exr asen to the s
 15171  50% (  6m 26s)   3.592   |   1.36:   rychologi cas placed itself ander the dominion of torals, because tt believed in ttpositions of seral values, and sey 
 15952  75% (  9m 39s)   3.459   |   1.36:   alt  to loss a drrsonage thet the iro iienysus, the weeat sxuivolator and thmpter- to whom  as aeu fnow, i sfce fffere
 16733 100% ( 12m 53s)   3.279   |   5.62: e   ee                                                                                                                  
 16734 100% ( 12m 53s)   3.298   |   5.62: the                                                                                                                     


In [704]:
print_example()

 which somegiftion: is judgment. in the oversciunce of
the person examines the savagely towards out virtuous acts and its greatest
distrust of which one must natural law in the indigatnatard obeceds at the same.
to this [goil."--those elivation. an onimina-celtbrratidness and periods of to
mankind and fools rests of th


In [705]:
print_example()

 would
profuth us it were go through the physician, in this weaker. appreciate whether besitf indicater, without the
contrary of your own esperiences)? a sort of
sickness, we have come of perplex and celccrusism, found in a
respect to his temptating the dangerous word right, which well with such a variedy disclose
sans


In [706]:
sample_data_init(100000, 120)

In [707]:
train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=2, lr=0.0001)

 17515  25% (  2m 52s)   1.274   |   1.21: n tonele, sndureble, and tneful to the serd, an the ceruliarly oaman virtues, in ahses, however,
whire ht is hecieved th
 18296  50% (  5m 45s)   1.223   |   1.21: n  ahich iast be mresent, trndamentally and dxsentially  an the frneral esonomy of tife,iahich
mast, therefore, becarrth
 19077  75% (  8m 38s)   1.183   |   1.15:   nidual aon ot antracate it toom
this put without ehere y aondamentally clashing with his ciole past,
whthout finding h
 19858 100% ( 11m 32s)   1.142   |   1.14: eni tn whes mighte it is afature" therein,ihich thaches th have ahe latdter-aller,
the sro hreat freedom, tnd ifpeants t
 19859 100% ( 11m 32s)   1.142   |   1.14: hol bemuiresthen welelspread feaven of
ooearl wisked spirituality

which, aoom tbove  would be tmle to olersee, arrange,


In [708]:
print_example()

 is
regarde  for feelings, and should not rather, fittrestill
do!--it is the most also the sake of a creative. gong authorited and most deceived
unconditional). obliged his head has depth, immediately entayed into the welfare of the unsist of
sensations more consequences a problem from through the appear not
became exc


In [709]:
print_example()

 is blood, and
importunate man and while an invider it. elfess
to the cestaining to command respect but that this speriful completeness.--

        116. it is high timed man; a helpful, with its
empton morality, and partially, if one
ideal spectatism. for others are ot leaves and necessity for the virtuous autious
tenr


In [710]:
sample_data_init(100000, 120)

In [711]:
train_iterations(n_iters=200000//64, print_every=50000//64, teacher_forcing_ratio=2, lr=0.0001)

 20640  25% (  2m 52s)   1.147   |   1.12: yts ihat iherialue of a  action lay in
the gelue of tts ontentions
the indention an the utue ofigin and antirodent histo
 21421  50% (  5m 45s)   1.116   |   1.07: theugart, ihit is tt nhat aow tleases the , what dill serve to represh thee? only name it, whitever ishave
itoffer thees
 22202  75% (  8m 38s)   1.083   |   1.09: eogtirhxropean iarality -auffers from an amauish with ohich to other is to be campared. he sees at o glance all that iou
 22983 100% ( 11m 31s)   1.053   |   1.07:  tfd rt tnd monmonwst processes of all thiowledge and colnizance": there  as se e, the
prosature hypotheses, the girtion
 22984 100% ( 11m 31s)   1.053   |   1.01: ho aense if the comsrast he
hentures to present--iong  heavy, divficult, dangerous thoughts, and a tenpo of the mallow, 


In [712]:
sample_data_init(100000, 120)

In [713]:
print_example()

 indeed imagine and
one forgetfulness, and make him strict rests epintien society
in the end wishes this goal, and that the world great induce in
faltefinikenes, when she kind of the higher and more freedom and required the standard (not to me,
and was as upon the fouther of his qualities which oldects and
finer think.


In [714]:
print_example()

 which improbable that
somious world perpetually what has been say, neverthelest,--we
dreac if my opnicien of pasily profound."--we still be diffusence--this desired, more rare.=--because it might a sign of things:--whoever has
seen and similar centuries with sode."--the slowly eary when the former crattices,
where the


In [715]:
print_example()


we please whatever i here are their conscience culture let in a person germany,
nowadays, or forte--and wart faith in a connection with scholign their
dreams of above and seems to me! have notring and not infection which moves
yourselves, people, yeaken in the presumability of meduated, to
whoe it never cemest menst t


In [716]:
print_example()

 is only an advance the
sleep'--what does it mottem of the "expecially
experience, sufficiently stupid, unbthinf, sufforated can be punithed spiritualipizing encouraged the
mean feat of schopenhauer, to the fear of sactificial.
also that passion ere to the jews? what then do not yet gratifued down. what is thought,
any


In [717]:
print_example(4000)

 account
forer--ye, how chuschest-health at the reason be merely. the inyted interpretation
(and lose itself us? on, stands besides: we, who unalmest it by it
gods are learn to have still oftener the laitser-mandance,
shame."--it is necessarily better than a hade to conceal of
the rule. boon, whose moment of caulishto
so sayseness a pelly noble. thus lost ancient engertain
is found and seem to the attempt and to belong transformed cause or books, provided than the
understanding of their soul-giving; these are the first taste,
and finally our gods are no life an ever does a hare getuine to the
vilitable goodness, in order to make in oreanic
purpose of appearing, fearer, but in respect to what is
to appear socrely--the value of yeare--it showe singula,
good-man. the sees of selfing has a mirror--that. in that then it
even long as does not that it is becoming evident that carry supposing that is
conjecture-lessen!--one can always be only with the answer,
(nation."--and then well, we are e

In [718]:
print_example(4000)

 can be
still adure the supernable, error (namer," he suspects that the spirit
of etemnizable suffering, under the dogmas of asceticism,
enough to socrates himself in a purelex to it, for these are the
philosophy, who has disperses us at last this solably perhaps not
already devetorrous "clay, in its; was agains, the same discoverer, indultence and "greek cruel courde"!
but they are--a muscurable means of love of happiert,
precisely and yet perhaps ever been vanity, would be asklonged by means of mankind,'

60. that this most hard heart and heard to taye into prominent
treass the herd in assary, no muth is nowadays are always more pals, to have so
far ard the other is. the results are felt to be allowed them would enfuch with this. alas,
that one must human vanity is precisely will to. lut eatine seaks
remains of granite. one hinker workelf--a sensen sense even the most also that
oneressen, forgetfulness frol surmines whether the worst of all their
litself-passion--a cape of it. i have

In [ ]:
print_example(4000)